# Bab 5: Support Vector Machines (SVMs)

### 1. Pendahuluan

Bab 5 memperkenalkan *Support Vector Machine* (SVM) sebagai model *Machine Learning* yang kuat dan serbaguna. SVM mampu melakukan klasifikasi linier atau non-linier, regresi, dan bahkan deteksi *outlier*. SVM sangat cocok untuk klasifikasi dataset yang kompleks dengan ukuran kecil hingga menengah. Bab ini akan menjelaskan konsep inti SVM, cara menggunakannya, dan bagaimana SVM bekerja di balik layar.

### 2. Klasifikasi SVM Linier (Linear SVM Classification)

Ide fundamental di balik SVM dijelaskan dengan mengilustrasikan dataset Iris yang dapat dipisahkan secara linier. Tujuan SVM adalah menemukan batas keputusan yang tidak hanya memisahkan kedua kelas, tetapi juga menjaga jarak sejauh mungkin dari *instance training* terdekat. Ini disebut *large margin classification*.

Penting untuk dicatat bahwa *instance training* yang berada "di luar jalan" (off the street) tidak akan memengaruhi batas keputusan sama sekali; batas keputusan sepenuhnya ditentukan oleh *instance* yang terletak di tepi jalan, yang disebut **vektor pendukung (support vectors)**.

SVM sensitif terhadap skala fitur. Melakukan penskalaan fitur (misalnya, menggunakan `StandardScaler` dari Scikit-Learn) sangat penting untuk mendapatkan batas keputusan yang lebih baik.

#### a. Klasifikasi Margin Lunak (Soft Margin Classification)

*Hard margin classification* adalah klasifikasi yang secara ketat mengharuskan semua *instance* berada di luar "jalan" dan di sisi yang benar. Namun, ini memiliki dua masalah utama: hanya berfungsi jika data dapat dipisahkan secara linier dan sensitif terhadap *outlier*.

Untuk mengatasi masalah ini, digunakan model yang lebih fleksibel yang disebut *soft margin classification*. Tujuannya adalah menyeimbangkan antara menjaga "jalan" sebesar mungkin dan membatasi pelanggaran margin (*margin violations*), yaitu *instance* yang berakhir di tengah jalan atau bahkan di sisi yang salah.

*Hyperparameter* `C` mengontrol *trade-off* ini. Nilai `C` yang rendah menghasilkan model dengan banyak pelanggaran margin tetapi kemungkinan generalisasi yang lebih baik, sedangkan nilai `C` yang tinggi membatasi pelanggaran margin. Jika model SVM mengalami *overfitting*, regularisasi dapat dilakukan dengan mengurangi `C`.

Contoh implementasi dengan Scikit-Learn menggunakan kelas `LinearSVC` ditunjukkan untuk mendeteksi bunga Iris virginica. Berbeda dengan pengklasifikasi Regresi Logistik, pengklasifikasi SVM tidak menghasilkan probabilitas untuk setiap kelas.

### 3. Klasifikasi SVM Non-Linier (Nonlinear SVM Classification)

Ketika data tidak dapat dipisahkan secara linier, ada beberapa pendekatan:

#### a. Menambahkan Fitur Polinomial
Salah satu pendekatan adalah menambahkan lebih banyak fitur, seperti fitur polinomial (misalnya, $x_2 = (x_1)^2$). Dalam beberapa kasus, ini dapat menghasilkan dataset yang dapat dipisahkan secara linier. Contoh dengan dataset *moons* (dua setengah lingkaran yang saling bersilang) dan `PolynomialFeatures` dari Scikit-Learn menunjukkan bagaimana ini bekerja.

#### b. Kernel Polinomial
Meskipun menambahkan fitur polinomial dapat efektif, pada derajat polinomial rendah mungkin tidak dapat menangani dataset yang sangat kompleks, dan pada derajat tinggi akan menghasilkan jumlah fitur yang sangat besar, membuat model terlalu lambat.

Untuk mengatasi ini, *kernel trick* dapat diterapkan pada SVM. *Kernel trick* memungkinkan SVM mencapai hasil yang sama seolah-olah banyak fitur polinomial telah ditambahkan, bahkan dengan polinomial derajat sangat tinggi, tanpa benar-benar menambahkannya. Ini diimplementasikan oleh kelas `SVC` di Scikit-Learn.

Contoh menggunakan `SVC` dengan `kernel="poly"` ditunjukkan. *Hyperparameter* `coef0` mengontrol seberapa besar model dipengaruhi oleh polinomial derajat tinggi dibandingkan polinomial derajat rendah. Jika model *overfitting*, derajat polinomial dapat dikurangi; jika *underfitting*, dapat ditingkatkan.

#### c. Fitur Kesamaan (Similarity Features)
Teknik lain untuk masalah non-linier adalah menambahkan fitur yang dihitung menggunakan fungsi kesamaan, yang mengukur seberapa mirip setiap *instance* dengan *landmark* tertentu. Fungsi *Gaussian Radial Basis Function* (RBF) adalah contoh fungsi kesamaan yang umum digunakan:
$$ \phi_{\gamma}(\mathbf{x}, \boldsymbol{\ell}) = \exp(-\gamma \| \mathbf{x} - \boldsymbol{\ell} \|^2) $$
Membuat *landmark* di lokasi setiap *instance* dalam dataset adalah pendekatan paling sederhana. Namun, ini dapat menghasilkan jumlah fitur yang sangat besar jika *training set* juga besar.

#### d. Kernel RBF Gaussian
Mirip dengan metode fitur polinomial, metode fitur kesamaan juga dapat mahal secara komputasi. Sekali lagi, *kernel trick* pada SVM memungkinkan hasil serupa tanpa perlu menghitung semua fitur tambahan. Contoh menggunakan `SVC` dengan `kernel="rbf"` ditunjukkan.
*Hyperparameter* `gamma` ($\gamma$) bertindak seperti *hyperparameter* regularisasi. Meningkatkan $\gamma$ membuat kurva berbentuk lonceng menjadi lebih sempit, sehingga batas keputusan lebih tidak beraturan (*overfitting*), sementara nilai $\gamma$ yang kecil membuat kurva lebih lebar, menghasilkan batas keputusan yang lebih halus (*underfitting*).

Penulis menyarankan untuk selalu mencoba *linear kernel* terlebih dahulu (`LinearSVC` lebih cepat dari `SVC(kernel="linear")`), terutama untuk dataset besar atau banyak fitur. Jika tidak terlalu besar, *Gaussian RBF kernel* juga direkomendasikan karena bekerja baik di sebagian besar kasus.

### 4. Kompleksitas Komputasi (Computational Complexity)

* **`LinearSVC`**: Berbasis library `liblinear`, yang memiliki kompleksitas waktu pelatihan sekitar $O(m \times n)$, di mana $m$ adalah jumlah *instance training* dan $n$ adalah jumlah fitur. Algoritma ini lebih lambat jika presisi sangat tinggi dibutuhkan (dikontrol oleh *hyperparameter* $\epsilon$ atau `tol`).
* **`SVC`**: Berbasis library `libsvm`, yang mendukung *kernel trick*. Kompleksitas waktu pelatihannya biasanya antara $O(m^2 \times n)$ dan $O(m^3 \times n)$. Ini membuatnya sangat lambat ketika jumlah *instance training* sangat besar (misalnya, ratusan ribu). Namun, `SVC` sangat baik untuk *training set* yang kompleks, berukuran kecil atau menengah, dan skalanya baik dengan jumlah fitur, terutama dengan fitur *sparse*.

Tabel perbandingan kelas klasifikasi SVM di Scikit-Learn disajikan:

| Class       | Time complexity          | Out-of-core support | Scaling required | Kernel trick |
| :---------- | :----------------------- | :------------------ | :--------------- | :----------- |
| `LinearSVC` | $O(m \times n)$          | No                  | Yes              | No           |
| `SGDClassifier` | $O(m \times n)$          | Yes                 | Yes              | No           |
| `SVC`       | $O(m^2 \times n)$ to $O(m^3 \times n)$ | No                  | Yes              | Yes          |

### 5. Regresi SVM (SVM Regression)

Algoritma SVM juga mendukung tugas regresi linier dan non-linier. Alih-alih mencoba menyesuaikan "jalan" terbesar di antara dua kelas, Regresi SVM mencoba menyesuaikan sebanyak mungkin *instance* ke dalam "jalan" sambil membatasi pelanggaran margin (*instance* di luar jalan). Lebar "jalan" dikontrol oleh *hyperparameter* $\epsilon$.
*Instance* yang berada di dalam margin tidak memengaruhi prediksi model, sehingga model dikatakan $\epsilon$-insensitif.

Kelas `LinearSVR` dari Scikit-Learn digunakan untuk Regresi SVM linier, dan kelas `SVR` digunakan untuk model SVM berbasis *kernel* untuk regresi non-linier. `LinearSVR` berskala linier dengan ukuran *training set*, sedangkan `SVR` menjadi sangat lambat untuk *training set* yang besar.

### 6. Di Balik Layar (Under the Hood)

Bagian ini menjelaskan secara lebih detail bagaimana SVM membuat prediksi dan bagaimana algoritma pelatihannya bekerja. Notasi parameter model diubah dari $\boldsymbol{\theta}$ menjadi $\mathbf{w}$ (vektor bobot fitur) dan $b$ (istilah bias).

#### a. Fungsi Keputusan dan Prediksi
Model pengklasifikasi SVM linier memprediksi kelas *instance* baru $\mathbf{x}$ dengan menghitung fungsi keputusan $\mathbf{w}^\intercal \mathbf{x} + b$. Jika hasilnya positif, kelas yang diprediksi $\hat{y}$ adalah kelas positif (1), jika tidak, kelas negatif (0).
$$ \hat{y} = \begin{cases} 0 & \text{if } \mathbf{w}^\intercal \mathbf{x} + b < 0, \\ 1 & \text{if } \mathbf{w}^\intercal \mathbf{x} + b \ge 0 \end{cases} $$
Batas keputusan adalah set titik di mana fungsi keputusan sama dengan 0. Garis putus-putus paralel mewakili titik di mana fungsi keputusan sama dengan 1 atau -1, membentuk margin. Melatih pengklasifikasi SVM linier berarti menemukan nilai $\mathbf{w}$ dan $b$ yang membuat margin selebar mungkin sambil menghindari atau membatasi pelanggaran margin. Semakin kecil vektor bobot $\mathbf{w}$, semakin besar marginnya.

#### b. Tujuan Pelatihan
Tujuan untuk *hard margin linear SVM classifier* dinyatakan sebagai masalah optimisasi terkendala:
$$ \min_{\mathbf{w}, b} \frac{1}{2} \mathbf{w}^\intercal \mathbf{w} \quad \text{subject to } t^{(i)}(\mathbf{w}^\intercal \mathbf{x}^{(i)} + b) \ge 1 \quad \text{for } i = 1, 2, \ldots, m $$
di mana $t^{(i)}$ adalah -1 untuk *instance* negatif dan 1 untuk *instance* positif. Kita meminimalkan $\frac{1}{2} \mathbf{w}^\intercal \mathbf{w}$ (setara dengan $\frac{1}{2} \| \mathbf{w} \|^2$) karena turunannya lebih sederhana.

Untuk tujuan *soft margin*, diperkenalkan variabel *slack* $\zeta^{(i)} \ge 0$ untuk setiap *instance*, yang mengukur seberapa banyak *instance* ke-$i$ diizinkan melanggar margin. *Hyperparameter* `C` mengontrol *trade-off* antara meminimalkan pelanggaran margin dan memperbesar margin.
Tujuan *soft margin linear SVM classifier*:
$$ \min_{\mathbf{w}, b, \boldsymbol{\zeta}} \frac{1}{2} \mathbf{w}^\intercal \mathbf{w} + C \sum_{i=1}^{m} \zeta^{(i)} \quad \text{subject to } t^{(i)}(\mathbf{w}^\intercal \mathbf{x}^{(i)} + b) \ge 1 - \zeta^{(i)} \quad \text{and } \zeta^{(i)} \ge 0 \quad \text{for } i = 1, 2, \ldots, m $$

#### c. Pemrograman Kuadratik (Quadratic Programming)
Masalah *hard margin* dan *soft margin* keduanya adalah masalah optimisasi kuadratik *convex* dengan kendala linier, dikenal sebagai masalah *Quadratic Programming* (QP). *Solver* QP siap pakai tersedia untuk menyelesaikannya.

#### d. Masalah Dual (The Dual Problem)
Setiap masalah optimisasi terkendala (*primal problem*) memiliki masalah lain yang terkait erat, yang disebut **masalah dual**. Solusi masalah dual biasanya memberikan batas bawah untuk solusi masalah primal, tetapi dalam kondisi tertentu, keduanya dapat memiliki solusi yang sama. Masalah SVM memenuhi kondisi ini, sehingga dapat diselesaikan baik dengan masalah primal maupun dual.

Bentuk dual dari tujuan SVM linier:
$$ \min_{\boldsymbol{\alpha}} \frac{1}{2} \sum_{i=1}^{m} \sum_{j=1}^{m} \alpha_i \alpha_j t^{(i)} t^{(j)} \mathbf{x}^{(i) \intercal} \mathbf{x}^{(j)} - \sum_{i=1}^{m} \alpha_i \quad \text{subject to } \alpha_i \ge 0 \quad \text{for } i = 1, 2, \ldots, m $$
Setelah menemukan vektor $\boldsymbol{\alpha}$ yang meminimalkan persamaan ini (menggunakan *solver* QP), $\mathbf{w}$ dan $b$ dapat dihitung. Masalah dual lebih cepat diselesaikan daripada masalah primal ketika jumlah *instance training* lebih kecil dari jumlah fitur. Yang terpenting, masalah dual memungkinkan *kernel trick*.

#### e. SVMs Berbasis Kernel (Kernelized SVMs)
*Kernel trick* memungkinkan penerapan transformasi polinomial atau kesamaan (seperti RBF) pada dataset secara implisit ke ruang berdimensi sangat tinggi (*feature space*), tanpa perlu secara eksplisit menghitung fitur-fitur tersebut. Ini membuat proses lebih efisien secara komputasi.
Fungsi kernel $K(\mathbf{a}, \mathbf{b})$ mampu menghitung *dot product* $\phi(\mathbf{a})^\intercal \phi(\mathbf{b})$ hanya berdasarkan vektor asli $\mathbf{a}$ dan $\mathbf{b}$.

Beberapa kernel umum:
* **Linear:** $K(\mathbf{a}, \mathbf{b}) = \mathbf{a}^\intercal \mathbf{b}$ 
* **Polynomial:** $K(\mathbf{a}, \mathbf{b}) = (\gamma \mathbf{a}^\intercal \mathbf{b} + r)^d$ 
* **Gaussian RBF:** $K(\mathbf{a}, \mathbf{b}) = \exp(-\gamma \| \mathbf{a} - \mathbf{b} \|^2)$ 
* **Sigmoid:** $K(\mathbf{a}, \mathbf{b}) = \tanh(\gamma \mathbf{a}^\intercal \mathbf{b} + r)$ 

*Mercer's Theorem* menyatakan bahwa jika fungsi $K(\mathbf{a}, \mathbf{b})$ memenuhi kondisi tertentu, maka ada fungsi $\phi$ yang memetakan $\mathbf{a}$ dan $\mathbf{b}$ ke ruang lain sehingga $K(\mathbf{a}, \mathbf{b}) = \phi(\mathbf{a})^\intercal \phi(\mathbf{b})$. Ini berarti kernel dapat digunakan tanpa perlu mengetahui atau menghitung $\phi$.

Untuk membuat prediksi dengan SVM berbasis kernel, rumus prediksi hanya melibatkan *dot product* antara vektor input baru dengan vektor-vektor pendukung:
$$ h_{\mathbf{w}, b}(\phi(\mathbf{x}_{\text{new}})) = \mathbf{w}^\intercal \phi(\mathbf{x}_{\text{new}}) + b = \sum_{i=1}^{m} \alpha_i t^{(i)} K(\mathbf{x}^{(i)}, \mathbf{x}_{\text{new}}) + b $$
Perhitungan $b$ juga melibatkan *kernel trick*.

#### f. SVMs Online (Online SVMs)
*Online SVM classifiers* dilatih secara inkremental, biasanya saat *instance* baru tiba. Salah satu metode untuk implementasi ini adalah menggunakan *Gradient Descent* (misalnya, `SGDClassifier`) untuk meminimalkan fungsi biaya yang berasal dari masalah primal.
Fungsi biaya pengklasifikasi SVM linier:
$$ J(\mathbf{w}, b) = \frac{1}{2} \mathbf{w}^\intercal \mathbf{w} + C \sum_{i=1}^{m} \max(0, 1 - t^{(i)}(\mathbf{w}^\intercal \mathbf{x}^{(i)} + b)) $$
Istilah $\max(0, 1 - t)$ disebut fungsi *hinge loss*.

### 7. Kesimpulan

Bab 5 berhasil menjelaskan SVM sebagai model yang sangat fleksibel untuk klasifikasi dan regresi, baik linier maupun non-linier. Konsep inti *large margin classification*, *soft margin*, dan *kernel trick* diuraikan dengan jelas, termasuk implikasinya terhadap kompleksitas komputasi. Pemahaman tentang masalah primal dan dual serta bagaimana *hinge loss* digunakan untuk pelatihan *online SVM* memberikan gambaran lengkap tentang prinsip kerja SVM.

## 1. Linear SVM Classification

### Generating some linearly separable data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

X = np.array([[1, 2], [1, 4], [1, 0], [4, 2], [4, 4], [4, 0]])
y = np.array([0, 0, 0, 1, 1, 1]) # Example labels

### Iris dataset example

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

### Soft Margin Classification using LinearSVC

In [3]:
from sklearn.svm import LinearSVC
# C is the regularization hyperparameter. Lower C means more regularization.
# loss="hinge" is for hard margin, loss="squared_hinge" is for soft margin.
# dual=True by default for LinearSVC. For large datasets, set to False.
# max_iter is for optimization algorithm, ensure it's high enough to converge.
lin_svc = LinearSVC(loss="hinge", C=1, random_state=42, max_iter=20000)
lin_svc.fit(X, y)

lin_svc.predict([[1.6, 0.5]]) # Example prediction
lin_svc.predict([[5.5, 1.7]])

array([1.])

### Using SGDClassifier (Linear SVM)

In [4]:
# SGDClassifier can also be used for linear SVMs with penalty="l2"
# loss="hinge" is the default for linear SVM
# Use a scaling factor to compare with LinearSVC C parameter
# For SGDClassifier, alpha controls regularization (opposite of C)
# eta0 is initial learning rate
# max_iter and tol for convergence
sgd_clf = SGDClassifier(loss="hinge", alpha=1/100, max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(X, y)

sgd_clf.predict([[5.5, 1.7]])

array([1.])

## 2. Nonlinear SVM Classification

### Polynomial Features

In [5]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42, max_iter=20000))
])

polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf',
                 LinearSVC(C=10, loss='hinge', max_iter=20000,
                           random_state=42))])

### Polynomial Kernel

In [7]:
from sklearn.svm import SVC
# kernel="poly" for polynomial kernel
# degree is the polynomial degree
# coef0 controls the influence of high-degree vs low-degree polynomials
# C is regularization hyperparameter
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5, random_state=42))
])
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly', random_state=42))])

### Gaussian RBF Kernel

In [8]:
# kernel="rbf" for Gaussian RBF kernel
# gamma is a hyperparameter for the RBF kernel
# C is regularization hyperparameter
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001, random_state=42))
])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5, random_state=42))])

## 3. SVM Regression

### Linear SVM Regression

In [9]:
# Generate some linear-looking data
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

from sklearn.svm import LinearSVR
# epsilon (epsilon) controls the width of the margin
# C is regularization hyperparameter
svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y.ravel()) # .ravel() is needed for y to be 1D

svm_reg.predict([[1.5]]) # Example prediction

array([8.45219347])

### Nonlinear SVM Regression

In [10]:
# Generate some nonlinear data (quadratic)
X = 2 * np.random.rand(100, 1) - 1
y = 0.2 + 0.1 * X + 0.5 * X**2 + np.random.randn(100, 1) / 10

from sklearn.svm import SVR
# kernel="poly" or "rbf" can be used for nonlinear regression
# degree for poly kernel, gamma for rbf kernel
# C and epsilon are also important
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y.ravel())

SVR(C=100, degree=2, kernel='poly')

In [11]:
svm_poly_reg.predict([[0.5]]) # Example prediction

array([0.33202023])

In [12]:
svm_rbf_reg = SVR(kernel="rbf", gamma=0.1, C=100, epsilon=0.1)
svm_rbf_reg.fit(X, y.ravel())

SVR(C=100, gamma=0.1)

In [13]:
svm_rbf_reg.predict([[0.5]]) # Example prediction

array([0.39314978])